## Experiment description

The goal of this experiment is to train a TM classifier trained on 40% of the data available in Wind Farm C, and then evaluate the model on the remaining 60% of the data. The model will be evaluated on the accuracy score.

In [1]:
# Import all required libraries

import numpy as np
import pandas as pd
import pickle
import os

from tmu.models.classification.vanilla_classifier import TMClassifier

2025-04-23 17:12:30,398 - tmu.util.cuda_profiler - WARNING - Could not import pycuda: No module named 'pycuda'
2025-04-23 17:12:30,398 - tmu.clause_bank.clause_bank_cuda - ERROR - No module named 'pycuda'
Traceback (most recent call last):
  File "/Users/kjellhaaland/Documents/GitHub/uia-master-thesis/.venv/lib/python3.12/site-packages/tmu/clause_bank/clause_bank_cuda.py", line 41, in <module>
    from pycuda._driver import Device, Context
ModuleNotFoundError: No module named 'pycuda'
2025-04-23 17:12:30,399 - tmu.clause_bank.clause_bank_cuda - WARNING - Could not import pycuda. This indicates that it is not installed! A possible fix is to run 'pip install pycuda'. Fallback to CPU ClauseBanks.


In [2]:
os.makedirs("models", exist_ok=True)
os.makedirs("logs", exist_ok=True)

In [3]:
wind_farm = "A"
train_datasets = np.array([68, 22, 72, 73, 0])
test_datasets = np.array([26, 40])

In [4]:
## Helper functions for saving the model and accuracy

# Helper function to save the model
def save_model(model, filename):
    with open(filename, "wb") as f:
        pickle.dump(model, f)


# Helper function to save the accuracy
def save_accuracy(epoch, accuracy, tp, tn, fp, fn):
    with open("logs/accuracy.txt", "a") as f:
        f.write(f"{epoch},{accuracy},{tp},{tn},{fp},{fn}\n")


def load_train_dataset(farm, event_id):
    X = np.loadtxt(f"./data_train/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X


def load_train_labels(farm, event_id):
    # Load dataframe from file
    df = pd.read_csv(f"./data_train/y_{farm}_{event_id}.csv")

    labels = df['label'].values
    status_ids = df['status_type_id'].values
    train_test = df['train_test'].values

    return np.array(labels).astype(np.uint32)  #, np.array(status_ids).astype(np.uint32), train_test


def load_test_dataset(farm, event_id):
    X = np.loadtxt(f"./data_test/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X


def load_test_labels(farm, event_id):
    # Load dataframe from file
    df = pd.read_csv(f"./data_test/y_{farm}_{event_id}.csv")

    labels = df['label'].values
    status_ids = df['status_type_id'].values
    train_test = df['train_test'].values

    return np.array(labels).astype(np.uint32)  #, np.array(status_ids).astype(np.uint32), train_test


def get_train_dataset():
    # Load all train datasets and concat them
    dataset = np.concatenate([load_train_dataset(wind_farm, dataset) for dataset in train_datasets])
    labels = np.concatenate([load_train_labels(wind_farm, dataset) for dataset in train_datasets])
    #labels = [x for (x, y, z) in labels]
    return dataset, labels


def get_test_dataset():
    # Load all train datasets and concat them
    dataset = np.concatenate([load_train_dataset(wind_farm, dataset) for dataset in test_datasets])
    labels = np.concatenate([load_train_labels(wind_farm, dataset) for dataset in test_datasets])
    #labels = [x for (x, y, z) in labels]
    return dataset, labels

In [5]:
# Number of features

(X_train, y_train) = get_train_dataset()
(X_test, y_test) = get_test_dataset()

print(f"Train dataset shape: {X_train.shape}")
print(f"Test dataset shape: {X_test.shape}")


Train dataset shape: (30000, 10000)
Test dataset shape: (12000, 2000)


In [ ]:
epochs = 100
platform = "CPU"

tm = TMClassifier(
    number_of_clauses=2000,
    T=800,
    s=15.0,
    max_included_literals=180,
    weighted_clauses=True,
    platform=platform,
    batch_size=250,
)

best_accuracy = 0

print(f"Running {TMClassifier} for {epochs} epochs")

for epoch in range(epochs):

    tm.fit(X_train, y_train)

    pred = tm.predict(X_test)

    pred_0s = np.count_nonzero(pred == 0)
    pred_1s = np.count_nonzero(pred == 1)

    print(f"Predicted 0s: {pred_0s}, Predicted 1s: {pred_1s}")

    accuracy = np.sum(pred == y_test) / len(y_test)

    # the number of false positives based on g and p
    fp = np.sum((pred == 1) & (y_test == 0))

    # the number of true negatives based on g and p
    tn = np.sum((pred == 0) & (y_test == 0))

    # the number of true positives based on g and p
    tp = np.sum((pred == 1) & (y_test == 1))

    # the number of false negatives based on g and p
    fn = np.sum((pred == 0) & (y_test == 1))

    print(f"Accuracy = {accuracy:.5f}, TP = {tp}, TN = {tn}, FP = {fp}, FN = {fn}")

    save_accuracy(epoch, accuracy, tp, tn, fp, fn)

    # Print every 20 epochs
    #if (epoch + 1) % 20 == 0:
    print(f"Epoch: {epoch + 1}, Accuracy: {accuracy:.5f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy

        print("Saving model")
        save_model(tm, "models/best.pkl")


Running <class 'tmu.models.classification.vanilla_classifier.TMClassifier'> for 100 epochs
